In [1]:
import numpy as np
import h5py
from glob import glob
from data_understanding import save_meta_data,read_meta_data
from tqdm import tqdm

In [2]:
id_class = {'building':0, 'vegetation':1, 'ground':2,'undefined':3}

In [3]:
metadata_json = glob("T**.json")

In [5]:
def append_xyz_rgb_class_ins_id(new_data,old_data):
    xyz = np.append(old_data[0],new_data[0],axis=0)
    rgb = np.append(old_data[1],new_data[1],axis=0)
    class_id = np.append(old_data[2],new_data[2])
    ins_id = np.append(old_data[3],new_data[3])
    return xyz,rgb,class_id,ins_id


In [6]:
def read_xyz_rgb_class_ins_id(class_,inst_ids_keys,index,metadata,path,new_data=None):
    if class_ == "building":
        xyz_rgb = np.loadtxt(metadata[class_][inst_ids_keys[index]][path])
    else: 
        xyz_rgb = np.loadtxt(f"Dataset/{metadata[class_][inst_ids_keys[index]][path]}")
    vec_size = xyz_rgb.shape[0]
    ins_id = np.zeros(vec_size,dtype="uint16")
    ins_id.fill(int(inst_ids_keys[index])) 
    class_id= np.zeros(vec_size,dtype="uint16")
    class_id.fill(int(id_class[class_]))
    if new_data:
        return append_xyz_rgb_class_ins_id(new_data,[xyz_rgb[:,:3],xyz_rgb[:,3:6].astype("int"),class_id,ins_id])
    return [xyz_rgb[:,:3],xyz_rgb[:,3:6].astype("int"),class_id.reshape(ins_id.shape[0],1),ins_id.reshape(ins_id.shape[0],1)]



In [7]:
def combine_all_class_to_area(building,vegetation,ground,undefined):
    return [np.vstack((building[0],vegetation[0],ground[0],undefined[0])),\
    np.vstack((building[1],vegetation[1],ground[1],undefined[1])),\
    np.vstack((
        np.array(building[2]).reshape(len(building[2]),1),
        np.array(vegetation[2]).reshape(len(vegetation[2]),1),
        np.array(ground[2]).reshape(len(ground[2]),1),
        np.array(undefined[2]).reshape(len(undefined[2]),1))),
    np.vstack((
        np.array(building[3]).reshape(len(building[3]),1),
        np.array(vegetation[3]).reshape(len(vegetation[3]),1),
        np.array(ground[3]).reshape(len(ground[3]),1),
        np.array(undefined[3]).reshape(len(undefined[3]),1)
        ))]

In [9]:
for area in tqdm(metadata_json):
    metadata = read_meta_data(area)
    for class_ in metadata.keys():
        # print(class_)
        if class_ == "building" :
            # continue
            inst_ids_keys = list(metadata[class_].keys())
            buff_data =  read_xyz_rgb_class_ins_id(class_="building",path="path",inst_ids_keys=inst_ids_keys,index=0,metadata=metadata)
            for idx in range(1,len(inst_ids_keys)):
                buff_data = read_xyz_rgb_class_ins_id(class_="building",path="path",inst_ids_keys=inst_ids_keys,index=idx,metadata=metadata,new_data=buff_data)
            building=buff_data
        if class_ == "vegetation" :
            # continue
            inst_ids_keys = list(metadata[class_].keys())
            buff_data =  read_xyz_rgb_class_ins_id(class_="vegetation",path="path",inst_ids_keys=inst_ids_keys,index=0,metadata=metadata)
            try:
                for idx in range(1,len(inst_ids_keys)):
                    buff_data = read_xyz_rgb_class_ins_id(class_="vegetation",path="path",inst_ids_keys=inst_ids_keys,index=idx,metadata=metadata,new_data=buff_data)
            except:
                pass
            vegetation=buff_data
        if class_ == "ground" :
            # continue
            inst_ids_keys = list(metadata[class_].keys())
            buff_data =  read_xyz_rgb_class_ins_id(class_="ground",path="path",inst_ids_keys=inst_ids_keys,index=0,metadata=metadata)
            try:
                for idx in range(1,len(inst_ids_keys)):
                    buff_data = read_xyz_rgb_class_ins_id(class_="ground",path="path",inst_ids_keys=inst_ids_keys,index=idx,metadata=metadata,new_data=buff_data)
            except:
                pass
            ground=buff_data
        if class_ == "undefined":
            # continue
            inst_ids_keys = list(metadata[class_].keys())
            buff_data =  read_xyz_rgb_class_ins_id(class_="undefined",path="path",inst_ids_keys=inst_ids_keys,index=0,metadata=metadata)
            try:
                for idx in range(1,len(inst_ids_keys)):
                    buff_data = read_xyz_rgb_class_ins_id(class_="undefined",path="path",inst_ids_keys=inst_ids_keys,index=idx,metadata=metadata,new_data=buff_data)
            except:
                pass
            undefined=buff_data
    xyz,rgb,class_id,ins_id= combine_all_class_to_area(building,vegetation,ground,undefined)
    buff = np.hstack((xyz,rgb,class_id.reshape((class_id.shape[0], 1)),ins_id.reshape((class_id.shape[0], 1))))
    np.savetxt(f"{area[:-5]}.txt",buff,fmt="%1.9f %1.9f %1.9f %d %d %d %d %d")
    f = h5py.File(f"{area[:-5]}.h5", 'w')
    f.create_dataset('xyz', data=xyz)
    f.create_dataset('colour', data=rgb)
    f.create_dataset('class_id', data=class_id)
    f.create_dataset('ins_id', data=ins_id)
    f.close()


  0%|          | 0/13 [00:00<?, ?it/s]